# Test Case LVV-T2219
The objective of this test case is to verify the ComCam communication interface with SAL by testing the commands, events and telemetry defined by the latest version of the XML. This test case will exercise the functionality of the ComCam on the 3rd level of the Summit and meets the following criteria:

    Only requires the ComCam to be operational
    Does not require any communication with other subsystems
    Does require the use of SAL and the EFD
    

The enumeration for the event substates and the fields published for each event/telemetry topic can be found in
https://ts-xml.lsst.io/sal_interfaces/CCCamera.html

In [ ]:
test_case = "LVV-T2219"
test_exec = "LVV-EXXXX"

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

# Setup

In [ ]:
import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst.ts.observatory.control.maintel import ComCam
from lsst_efd_client import EfdClient
from lsst.ts import salobj

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

Setting up logger

In [ ]:
logger = logging.getLogger(test_case)
logger.setLevel(logging.DEBUG)

Make sure DDS Daemon is running and startup Domain

In [ ]:
domain = salobj.Domain()

Start script task

In [ ]:
script = salobj.Controller("Script", index=vandv.get_index(test_case))
await asyncio.sleep(10) 

script.start_task

EFD setup

In [ ]:
client = vandv.efd.create_efd_client()

ComCam Initialization

In [ ]:
comcam = ComCam(domain=domain)
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

Helper functions: Event callback and telemetry check

In [ ]:
def evt_time_callback(evt):
    """Print event and time when callback event is received."""
    evt_time = Time(evt.private_sndStamp, format="unix", scale="tai")
    evt_time.format = "iso"
    evt_name = str(evt.__class__).split('_')[2]
    logger.info(f'\n {evt_name} logevent at {evt_time} is \n \t{evt}')

In [ ]:
async def check_efd(topic):
    """ Prints the last EFD published telemetry/event """
    try:
        tel = await client.select_top_n(topic,'*',num=1)
        tel_time = Time(tel.private_efdStamp, format="unix", scale="utc")
        tel_time.format = "iso"
        logger.info(f'Last {topic} telemetry at {tel_time[0]} UTC (time now {Time.now()}) was \n\n {tel}')
        
    except: 
        logger.info(f'{topic} was NOT found at EFD') 

Publish in the EFD that test is starting

In [ ]:
script.log.info(f'START- {test_case} -- at {Time(datetime.now())}')

# Start Up

--- 
## 1: ComCam Startup Procedure
LVV-T2239 (1.0) ComCam Startup Procedure

---
## 2: Available Filters 
Verify the CCCamera_logevent_availableFilters event is published to the EFD. 
Note: During the successful start up of the system, this event will be published automatically.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_availableFilters')

---
## 3: CCS Command State 
Initially verify the CCCamera_logevent_ccsCommandState event is publishing to the EFD. 

As there are no commands at the moment, the CCCamera_logevent_ccsCommandState event publishes IDLE.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_ccsCommandState')

---
## 4: Enable Calibration 
While the ccsCommandState is in IDLE, send a CCCamera_command_enableCalibration command.

Note: By default, the CalibrationDetailedState should be in DISABLED. If the CalibrationDetailedState is already in ENABLED, this command will not be accepted.

    Results:
    The CCCamera_logevent_ccsCommandState event publishes BUSY
    The CCCamera_logevent_calibrationDetailedState publishes ENABLED.
    The CCCamera_logevent_ccsConfigured event is published to the EFD noting the time at which the state transition occurred. 
    
<b>**** This CCCamera_logevent_ccsConfigured event does not exist, use instead comcam.rem.cccamera.logevent_configurationApplied<b>

In [ ]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_ccsCommandState.callback = evt_time_callback   

In [ ]:
# Enable Calibration
await comcam.rem.cccamera.cmd_enableCalibration.set_start()

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_configurationApplied')

In [ ]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None
comcam.rem.cccamera.evt_ccsCommandState.callback = None   

---
## 5: Clear Command
While the CalibrationDetailedState is in ENABLED,, send a CCCamera_command_clear command.

    Deviation: The CCCamera_command_clear no longer operational as part of the latest XML release (v12.0.0).
    The detector is automatically cleared every time exposure is taken. Therefore, no separate clearing is needed and the   CCCamera_logevent_RaftsDetailedState is not expected to be published as a result of this command. This step was left in order to verify that the CCCamera_command_clear will not do anything. The CLEARING state will show up after an exposure is taken (See the Image Test below).


    Results:
    The command is accepted but publishes no events    

In [ ]:
cmd = await comcam.rem.cccamera.cmd_clear.set_start(nClears=2)
logger.info(cmd)

---
## 6: Focal Plane Summary Info
Verify the CCCamera_logevent_focalPlaneSummaryInfo event is published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_focalPlaneSummaryInfo')

# Image Test

---
## 7: Start Image
While the CalibrationDetailedState is in ENABLED, send a CCCamera_command_startImage command with the following parameters:

                shutter: True
                sensors
                keyValueMap:
                config/requiredKeywords = [groupId, imageType]
                obsNote: Any arbitrary string
                timeout: 30s

    Results:  
    The CCCamera_logevent_calibrationDetailedState event publishes INTEGRATING
    The CCCamera_logevent_shutterDetailedState event publishes to the EFD updating froom CLOSED to OPEN
    A CCCamera_logevent_endShutterOpen event is published at the time the shutter opens
    A CCCamera_logevent_startReadout event is published
    The CCCamera_logevent_RaftsDetailedState event published to the EFD shows it is in the CLEARING state.
    
<b> Beware: The next two steps (discardRows and endImage) need to be run while the camera is INTEGRATING (30 s timeout)<b>

In [ ]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_shutterDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_raftsDetailedState.callback = evt_time_callback

In [ ]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [ ]:
# Set parameters
comcam.rem.cccamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=30, obsNote='Test')

In [ ]:
# Start image
await comcam.rem.cccamera.cmd_startImage.start()

----
## 8: Discard Rows 
While the CalibrationDetailedState is in INTEGRATING, send a CCCamera_command_discardRows command of 2 rows.

In [ ]:
cmd = await comcam.rem.cccamera.cmd_discardRows.set_start(2)
logger.info(cmd)

--- 
## 9: End Image 
Before the specified timeout, send a CCCamera_command_endImage command.

Note: If the CCCamera_command_endImage command isn't sent before the specified timeout parameter of the previous command, the shutter will close and the command will not be accepted.

    Results:
    The shutters close 
    A CCCamera_logevent_imageReadoutParameters event is published
    The CCCamera_logevent_shutterDetailedState event publishes to the EFD updating froom OPEN to CLOSED
    A CCCamera_logevent_endShutterClose event is published at the time the shutter close
    A CCCamera_logevent_endReadout event is published

In [ ]:
comcam.rem.cccamera.evt_imageReadoutParameters.callback = evt_time_callback
comcam.rem.cccamera.evt_shutterDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_endImage.start()

In [ ]:
comcam.rem.cccamera.evt_imageReadoutParameters.callback = None
comcam.rem.cccamera.evt_shutterDetailedState.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

In [ ]:
# Closing callbacks from startImage.
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None
comcam.rem.cccamera.evt_shutterDetailedState.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_raftsDetailedState.callback = None

# Set Filter Tests

---
## 10: Set Filter
Send a CCCamera_command_setFilter command for positionFilter1.

Note: The filter being set must be one that was returned by the CCCamera_logevent_availableFilters event and cannot be the filter that is already set.
    
    Results:
    The CCCamera_logevent_startSetFilter event is published to the EFD with the name of the filter being set
    The CCCamera_logevent_startLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing UNLOADED
    The CCCamera_logevent_endLoadFilter event is published to the EFD
    The CCCamera_logevent_startRotateCarousel event is published to the EFD
    The CCCamera_logevent_endRotatorCarousel event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD and showing LOADING
    The CCCamera_logevent_startLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing LOADED
    The CCCamera_logevent_endSetFilter event is published to the EFD

In [ ]:
await comcam.get_current_filter()

In [ ]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startLoadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endLoadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startRotateCarousel.callback = evt_time_callback
comcam.rem.cccamera.evt_endRotateCarousel.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_setFilter.set_start(name = 'i_06')

In [ ]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_startLoadFilter.callback = None
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = None
comcam.rem.cccamera.evt_endLoadFilter.callback = None
comcam.rem.cccamera.evt_startRotateCarousel.callback = None
comcam.rem.cccamera.evt_endRotateCarousel.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

In [ ]:
await comcam.get_current_filter()

---
## 11: Set Filter to None 
Send a CCCamera_command_setFilter command with NONE.

Note: This indicates that the loaded filter should be unloaded. However, this will only work if one of the filter positions is EMPTY. If there are no filter positions that are empty, we expect this step to fail.

    Results:
    The CCCamera_logevent_startSetFilter event is published to the EFD with the name of the filter being set
    The CCCamera_logevent_startUnloadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing UNLOADING
    The CCCamera_logevent_endUnloadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD 
    The CCCamera_logevent_endSetFilter event is published to the EFD

In [ ]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startUnloadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endUnloadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_setFilter.set_start(name = 'None')

In [ ]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_startUnloadFilter.callback = None
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = None
comcam.rem.cccamera.evt_endUnloadFilter.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

# Take Image test

--- 
## 12: Init Image
While in the ENABLED state, send a CCCamera_command_initImage command of 15s.

Note: This deltaT value is an estimate of time before sending the CCCamera_command_takeImage command. If the command can be sent sooner/later, adjust the deltaT value.

    Results: 
    The command is accepted
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Getting_Ready state
    A CCCamera_logevent_prepareToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_NotReadyToTakeImage event is published before reaching the Ready state.
    
    
<b> **** Not available at EFD `comcam.rem.cccamera.evt_prepareToTakeImage` & `comcam.rem.cccamera.evt_notReadyToTakeImage` <b>

In [ ]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_prepareToTakeImage.callback = evt_time_callback
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_initImage.set_start(deltaT = 15)

In [ ]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = None
comcam.rem.cccamera.evt_prepareToTakeImage.callback = None
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = None

---
##  13: Ready to take image.  
Verify the Camera is ready to take a picture.
    
    Results:
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Ready state
    A CCCamera_logevent_readyToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_endInitializeImage event is published to the EFD at the time of the transition
    
<b> **** Not available at EFD `comcam.rem.cccamera.evt_readyToTakeImage` & `comcam.rem.cccamera.evt_endInitializeImage` <b>

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_imageReadinessDetailedState')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_readyToTakeImage')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endInitializeImage')

---
## 14: Take Images
With the Camera ready to take a picture, send the CCCamera_command_takeImages command.

    Results:
    A CCCamera_logevent_startIntegration event is published to the EFD
    A CCCamera_logevent_startShutterOpen event is published to the EFD.
    A CCCamera_logevent_endShutterOpen event is published to the EFD.
    The Camera publishes a CCCamera_logevent_startShutterClose event as the camera shutter starts to shut
    The Camera publishes a CCCamera_logevent_endShutterClose event when the camera shutter is closed
    A CCCamera_logevent_startReadout event is published
    A CCCamera_logevent_endReadout event is published

In [ ]:
comcam.rem.cccamera.evt_startIntegration.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [ ]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [ ]:
# Set parameters
comcam.rem.cccamera.cmd_takeImages.set(shutter=True, keyValueMap=keyValueMap, numImages=1, expTime=20, obsNote = 'Test')

In [ ]:
await comcam.rem.cccamera.cmd_takeImages.start()

In [ ]:
comcam.rem.cccamera.evt_startIntegration.callback = None
comcam.rem.cccamera.evt_startShutterOpen.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startShutterClose.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

---
## 15: Camera is done taking images 
Verify the Camera is done taking pictures.

    Results:
    A CCCamera_logevent_endTakeImage event is published to the EFD at the time the state transition occurs.
    A CCCamera_logevent_endOfImageTelemetry event is published
    
<b> **** Not available at EFD `comcam.rem.cccamera.evt_endTakeImage`  <b>

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endTakeImage')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endofImageTelemetry')

# Guiders Image Test

---
## 16: Init Image

While in the ENABLED state, send a CCCamera_command_initImage command of 15s.

    Results:
    The command is accepted
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Getting_Ready state
    A CCCamera_logevent_prepareToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_NotReadyToTakeImage event is published before reaching the Ready state.

In [ ]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_prepareToTakeImage.callback = evt_time_callback
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_initImage.set_start(deltaT = 15)

In [ ]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = None
comcam.rem.cccamera.evt_prepareToTakeImage.callback = None
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = None

---
## 17: Ready to take image 
Verify the Camera is ready to take a picture.

    Results:
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Ready state
    A CCCamera_logevent_readyToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_endInitializeImage event is published to the EFD at the time of the transition

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_imageReadinessDetailedState')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_readyToTakeImage')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endInitializeImage')

---
## 18: Init Guider 
Send a CCCamera_command_initGuider command.

Note: Guider Implementation is not complete and so the use of this command at the moment will result in a fail.

    Results:
    An CCCamera_logevent_endInitializeGuider event is published to the EFD.

<b> **** Missing at EFD `comcam.rem.cccamera.evt_endInitializeGuider` <b>

In [ ]:
comcam.rem.cccamera.evt_endInitializeGuider.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_initGuiders.set_start()

In [ ]:
comcam.rem.cccamera.evt_endInitializeGuider.callback = None

---
## 19: Take Images
With the Camera ready to take a picture, send the CCCamera_command_takeImages command.

    Results:
    A CCCamera_logevent_startIntegration event is published to the EFD
    A CCCamera_logevent_startShutterOpen event is published to the EFD.
    A CCCamera_logevent_endShutterOpen event is published to the EFD.
    The Camera publishes a CCCamera_logevent_startShutterClose event as the camera shutter starts to shut
    The Camera publishes a CCCamera_logevent_endShutterClose event when the camera shutter is closed
    A CCCamera_logevent_startReadout event is published
    A CCCamera_logevent_endReadout event is published

In [ ]:
comcam.rem.cccamera.evt_startIntegration.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [ ]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [ ]:
# Set parameters
comcam.rem.cccamera.cmd_takeImages.set(shutter=True, keyValueMap=keyValueMap, numImages=1, expTime=20, obsNote= 'test')

In [ ]:
await comcam.rem.cccamera.cmd_takeImages.start()

In [ ]:
comcam.rem.cccamera.evt_startIntegration.callback = None
comcam.rem.cccamera.evt_startShutterOpen.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startShutterClose.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

---
## 20: Camera is done taking images 
Verify the Camera is done taking pictures.

    Results:
    A CCCamera_logevent_endTakeImage event is published to the EFD at the time the state transition occurs.
    A CCCamera_logevent_endOfImageTelemetry event is published

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endTakeImage')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endofImageTelemetry')

# Check Events in EFD

---
## 21: Image Visualization
Verify the CCCamera_logevent_imageVisualization event is not published into the EFD.    

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_imageVisualization')

---
## 22: Image Stored 
Verify the CCCamera_logevent_imageStored event is not published into the EFD.


Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_imageStored')

# Stop Command

---
## 23: Set Filter


While in the ENABLED state, send CCCamera_command_setFilter command for positionFilter1.
    
    Results:
    The CCCamera_logevent_startSetFilter is published to the EFD with the name of the filter being set
    
    
The next step `24: Stop` should be run right after this one, so the first `setFilter` command can be stopped as it'll be run in the BG. 

In [ ]:
await comcam.get_current_filter()

In [ ]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [ ]:
comcam.rem.cccamera.cmd_setFilter.set_start(name = 'r_03')

---
## 24: Stop 
Before an CCCamera_logevent_endSetFilter event is published, send a CCCamera_command_stop command.

Note: The CCCamera_command_setFilter command will stop once it is safe to do so. The CCCamera_command_stop command may not stop the filter from being set if the command was given while in an unsafe position.

    Results:
    The command is accepted
    The CCCamera_logevent_endSetFilter event shows that no filter has been set

In [ ]:
comcam.rem.cccamera.cmd_stop.set_start()

In [ ]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

In [ ]:
await comcam.get_current_filter()

---
## 25: Start Image
While the CalibrationDetailedState is in ENABLED, send a CCCamera_command_startImage command with the following parameters:

                        shutter: True
                        sensors
                        keyValueMap
                        obsNote: Any arbitrary string
                        timeout: 30s

The next step `26: Stop` should be run right after this one, so the first `setFilter` command can be stopped as it'll be run in the BG. 

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_calibrationDetailedState')

In [ ]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [ ]:
# Set parameters
comcam.rem.cccamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=30, obsNote='Test')

In [ ]:
# Start image
await comcam.rem.cccamera.cmd_startImage.start()

---
## 26: Stop 
Before the specified timeout, send a CCCamera_command_stop command.

    Results:
    The previous CCCamera_command_startImage command is abandoned.

In [ ]:
# Start integration event
await check_efd('lsst.sal.CCCamera.logevent_startIntegration')

In [ ]:
stop = comcam.rem.cccamera.cmd_stop.set_start()
logger.info(f'{stop} at {Time.now()}')

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_endOfImageTelemetry')

# Playlist Test - only valid when using emulated DAQ

---
## 27: Create playlist
Send a CCCamera_command_playList command of the playlist to be defined and the current folder of where the images exist.

    Results:
    The command is accepted and a playlist is created.
    
See more details on the syntax to create a playlist 
https://ts-xml.lsst.io/sal_interfaces/CCCamera.html 

In [ ]:
await comcam.rem.cccamera.cmd_playlist.set_start(playlist='test.playlist', folder =  , images = :)

---
## 28: Play playlist in repeat 
Send a CCCamera_command_play coommand of the playlist and an input of true for the repeat parameter.
    
    Results:
    The command is accepted
    The playlist is played and repeats

In [ ]:
await comcam.rem.cccamera.cmd_play.set_start(playlist='tiago-1.playlist', repeat =True)

# Disable Calibration 

---
##  29: Disable Calibraiton
With the Summary State of the Camera in ENABLED and the CalibrationDetailedState in ENABLED, send a CCCamera_command_disableCalibration command.

    Results:
    The  CCCamera_logevent_CalibrationDetailedState event publishes DISABLED.

In [ ]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback

In [ ]:
await comcam.rem.cccamera.cmd_disableCalibration.set_start()

In [ ]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None

--- 
# EFD Events

---
## 31: Shutter Blade Motion Profile Event
Verify the CCCamera_logevent_ShutterBladeMotionProfile event is not published to the EFD.

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.evt_shutterBladeMotionProfile')

---
## 32: Fits Files Written Event
Verify the CCCamera_logevent_fitsFilesWritten event is not published to the EFD.

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_fitsFilesWritten')

---
## 32: File Command Execution Event
Verify the CCCamera_logevent_FileCommandExecution event is not published to the EFD.

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.logevent_FileCommandExecution')

# EFD Telemetry

---
## 33: FCS
Verify the CCCamera_fcs telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.fcs')

---
## 34: Bonn Shutter Device
Verify the CCCamera_bonn_shutter_Device telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.bonn_shutter_Device')

---
## 35: DAQ Monitor Store
Verify the CCCamera_daq_monitor_Store telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.daq_monitor_Store')

---
## 36: Rebpower REB
Verify the CCCamera_rebpower_Reb telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.rebpower_Reb')

---
## 37: Rebpower REBPS 
Verify the CCCamera_rebpower_Rebps telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.rebpower_Rebps')

---
## 38: Vacumm Cold1
Verify the CCCamera_vacuum_Cold1 telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_Cold1')

---
## 39: Vacuum Cold2 
Verify the CCCamera_vacuum_Cold2 telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_Cold2')

---
## 40: Vacuum Cryo 
Verify the CCCamera_vacuum_Cryo telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_Cryo')

---
## 41: Vacuum IonPumps
Verify the CCCamera_vacuum_IonPumps telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_IonPumps')

---
## 42: Vacuum RTDS
Verify the CCCamera_vacuum_Rtds telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_Rtds')

---
## 43: Vacuum Turbo
Verify the CCCamera_vacuum_Turbo telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_Turbo')

---
## 44: Vacuum VQMonitor
Verify the CCCamera_vacuum_VQMonitor telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.vacuum_VQMonitor')

---
## 45: Quadbox BFR
Verify the CCCamera_quadbox_BFR telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.quadbox_BFR')

---
## 46: Quadbox PDU 24VD 
Verify the CCCamera_quadbox_PDU_24VD telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.quadbox_PDU_24VD')

---
## 47: Quadbox PDF 24VDC
Verify the CCCamera_quadbox_PDU_24VDC telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.quadbox_PDU_24VC')

---
## 48: Quadbox PDU 48V 
Verify the CCCamera_quadbox_PDU_48V telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.quadbox_PDU_48V')

---
## 49: Quadbox PDU 5V 
Verify the CCCamera_quadbox_PDU_5V telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.quadbox_PDU_5V')

---
## 50: Focal Place CCD 
Verify the CCCamera_focal_plane_Ccd telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.focal_plane_Ccd')

---
## 51: Focal Plane Reb
Verify the CCCamera_focal_plane_Reb telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.focal_plane_Reb')

---
## 52: Focal Plane Reb Total Power
Verify the CCCamera_focal_plane_RebTotalPower telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.focal_plane_RebTotalPower')

---
## 53: Focal Plane Segment
Verify the CCCamera_focal_plane_Segment telemetry data is being published to the EFD.

In [ ]:
await check_efd('lsst.sal.CCCamera.focal_plane_Segment')

In [ ]:
script.log.info(f"STOP - {test_case}")